# Working with processes and jobs
Unity helps teams move quickly from algorithm development and testing in Jupyter into large-scale processing with the Scaled Processing System (SPS). In order to do this, algorithms developed in Jupyter must be: 
1. Registered in the Application Catalog (see 1_working_with_applications.ipynb) as _Applications_, 
2. Deployed to an Application Deployment and Execution Service (ADES) in SPS where they are referred to as _Processes_, and then may be 
3. Run on the SPS as _Jobs_.

The intent of this tutorial is to help familiarize yourself with execution of code at scale using _Processes_ and _Jobs_. Unity provides users the ability to execute _Jobs_ to produce data. The steps below will showcase how a job is typically submitted to the Unity Platform.

## 0. Set up imports, predefined variables, and authentication

For this Tutorial we will make use of the Unity-Py client Python package.

### Imports

In [6]:
import requests
import time

from datetime import datetime
from IPython.display import JSON

from unity_sds_client.unity import Unity
from unity_sds_client.unity_services import UnityServices
from unity_sds_client.resources.job_status import JobStatus

### Predefined Variables

In [7]:
unity = Unity()
unity.set_venue_id("unity-sips-test")
process_service = unity.client(UnityServices.PROCESS_SERVICE)

Please enter your Unity username:  gangl
Please enter your Unity password:  ········


## Listing all deployed processes

The Unity-Py client provides the ability to view all deployed application packages (a.k.a., Processes) on the system using the `get_processes` function. After a successful deployment of an application package to SPS, you should see a new entry for the deployed application.

The `id` property is one of the properties needed to execute the Process and see existing Jobs for a given Process.

In [8]:
processes = process_service.get_processes()
for process in processes:
    print("Process ID: {}".format(process.id))
    print("Process Title: {}".format(process.title))
    print("Process Abstract: {}".format(process.abstract))
    print("Process Version: {}".format(process.process_version))
    print("")
    print(process)
    
my_process = processes[0]


Process ID: chirp:develop
Process Title: CHIRP Rebinning
Process Abstract: chirp
Process Version: develop

unity_sds_client.resources.Process(
    id="chirp:develop",
    title="CHIRP Rebinning",
    abstract="chirp",
    execution_unit="1",
    immediate_deployment=true,
    job_control_options=['async-execute'],
    keywords="",
    output_transmission=['reference'],
    ows_context_url="https://raw.githubusercontent.com/unity-sds/unity-sps-workflows/mike-gangl-patch-1/sounder_sips/chirp/chirp-rebinning-e2e-workflow-wrapper.test.cwl",
    process_version="develop"
)


### Retrieve a deployed process's information

If you know the ID of the process, the process information can be retrieved using the `get_process(id)` method of `ProcessService`.


In [9]:
process = process_service.get_process('chirp:develop')
print(process)

unity_sds_client.resources.Process(
    id="chirp:develop",
    title="CHIRP Rebinning",
    abstract="chirp",
    execution_unit="1",
    immediate_deployment=true,
    job_control_options=['async-execute'],
    keywords="",
    output_transmission=['reference'],
    ows_context_url="https://raw.githubusercontent.com/unity-sds/unity-sps-workflows/mike-gangl-patch-1/sounder_sips/chirp/chirp-rebinning-e2e-workflow-wrapper.test.cwl",
    process_version="develop"
)


## Execute a job
Before deploying Applications and working with jobs, it is assumed that a system administrator has deployed an ADES. These are often called "venues" or "environments", and some examples may be dev, test, prod, etc. To run a Job, you need a Process available as well (a deployed Application). In this case we have a demo application deployed and referenced in the setup step 0 above.

With an ADES and a Process ready to accept our Job, we can submit a Job along with any input data or parameters that are needed (as defined by a template job definition for a particular Application). In this example case, none are needed so `inputs` is blank. The response will provide a Job ID that we will store in a variable called `job_id` for use later.


In [10]:
data = {
  "mode": "async",
  "response": "document",
  "inputs": [
    {
      "id": "input_processing_labels",
      "data": [
        "gangl_test"
      ]
    },
    {
      "id": "input_cmr_collection_name",
      "data": "C2011289787-GES_DISC"
    },
    {
      "id": "input_cmr_search_start_time",
      "data": "2016-08-30T00:10:00Z"
    },
    {
      "id": "input_cmr_search_stop_time",
      "data": "2016-08-31T01:10:00Z"
    },
    {
      "id": "input_cmr_edl_user",
      "data": "cmr_user"
    },
    {
      "id": "input_cmr_edl_pass",
      "data": "cmr_pass"
    },
    {
      "id": "output_collection_id",
      "data": "urn:nasa:unity:ssips:TEST1:CHRP_16_DAY_REBIN___1"
    },
    {
      "id": "output_data_bucket",
      "data": " ssips-test-ds-storage-reproc"
    },
    {
      "id": "input_daac_collection_shortname",
      "data": "CHIRP_L1B"
    },
    {
      "id": "input_daac_collection_sns",
      "data": "arn:://SNS-arn"
    }
  ],
  "outputs": [
    {
      "id": "output",
      "transmissionMode": "reference"
    }
  ]
}

try:
    unity = Unity()
    unity.set_venue_id("unity-sips-test")
    process_service = unity.client(UnityServices.PROCESS_SERVICE)
    my_process = process_service.get_process('chirp:develop')
    # Store Job ID to use in future steps
    job = my_process.execute(data)
    print(job)

    # If the job submission is successful, print a success message along with the returned JOB-ID
    print("\nJob Submission Successful!\nJOB ID: {}\n".format(job.id))

except requests.exceptions.HTTPError as e:
    # An error has occurred, print the error message that was generated
    print(e.reason)

Please enter your Unity username:  gangl
Please enter your Unity password:  ········


unity_sds_client.resources.Job(
    id="3cac81ed-2727-4b5c-9f26-5e50654135a2",
    status="",
    inputs=None
)

Job Submission Successful!
JOB ID: 3cac81ed-2727-4b5c-9f26-5e50654135a2



## Get the job status / Monitor Job
The code below will demonstrate how one can check the status of the Job. The potential status responses are documented [here] _need a reference to process status_.

In this example, we will look up the status of the predfined application name from the setup step 0, and the job ID that was returned previously. This function will loop/block until the process ends. You will see a printout every 5 seconds.

In [10]:
try:

    job_status = job.get_status()
    
    while job_status == JobStatus.RUNNING:
        print("Status for job \"{}\" ({}): {}".format(job.id, datetime.now().strftime("%H:%M:%S"), job_status.value))
        time.sleep(5)
        job_status = job.get_status()
    
    # Print the job status
    print("\nStatus for job \"{}\" ({}): {}\n".format(job.id, datetime.now().strftime("%H:%M:%S"),job_status.value))
    
except requests.exceptions.HTTPError as e:
    # An error has occurred, print the error message that was generated
    print(e.reason)



Status for job "17df459d-ac85-45f6-800c-94a09a145f8f" (22:17:42): accepted



In [11]:
my_process

## List all jobs for a given process

What if I restarted my notebook and I have no Job ID? The API can ask the process endpoint to list the Jobs for a given Process (deployed Application).


In [14]:
print("Jobs running for Process: ", my_process.id, "\n")

jobs = process_service.get_jobs(my_process)

for job in jobs:
    print(job)

Jobs running for Process:  chirp:develop 

unity_sds_client.resources.Job(
    id="3cac81ed-2727-4b5c-9f26-5e50654135a2",
    status="running",
    inputs=None
)
unity_sds_client.resources.Job(
    id="69d4e1a0-c895-488b-b95d-e88e3facafa2",
    status="succeeded",
    inputs=None
)
unity_sds_client.resources.Job(
    id="1d47a125-8707-4ec7-bbea-7b8d590a0bb4",
    status="failed",
    inputs=None
)
unity_sds_client.resources.Job(
    id="5826ad28-cd16-4366-ac85-b8509169403c",
    status="succeeded",
    inputs=None
)
unity_sds_client.resources.Job(
    id="533ad8c2-f17a-4fb2-8d05-591740a05efc",
    status="failed",
    inputs=None
)
unity_sds_client.resources.Job(
    id="44668746-cade-48de-9c86-4c647032cc3c",
    status="succeeded",
    inputs=None
)
unity_sds_client.resources.Job(
    id="44242c23-48a1-47b4-b244-e856743cd61d",
    status="succeeded",
    inputs=None
)
unity_sds_client.resources.Job(
    id="48b087b5-aff2-4ca0-87ff-4b156d2a5f93",
    status="succeeded",
    inputs=None

# Following Sections are In Progress

## Get job results
Now that we've monitored the status of a Job, and verified that is has completed, we can retreive information about where the generated data is located. 

In this example, we will use the predefined Process name, and the Job ID that was returned previously.

## So... where are the results?

Currently the results are not made available to the endpoint, but a future release will include a STAC document of generated files and their locations. For now, the results are written to the `SNDR_SNPP_ATMS_L1B_OUTPUT___1` product type in the Unity system. Again, future releases will allow the specification of the output collection (by name or version).

## Dismiss a job

What if after a job was started, it was determined that it is no longer needed? The job can be dismissed as shown in the example below so long as it is still running.

In [ ]:
print("{},{}".format(my_process.id, job.id))
response = job.dismiss()

print("Response: ", response, "\n")